In [ ]:
from data_loader import DataLoader
from data_inspector import DataInspector
from data_filter import DataFilter

In [2]:
data_dir = "/Users/ahenry/Documents/toolboxes/wind_forecasting/examples/data"
file_signature = "kp.turbine.z02.b0.202203*.wt073.nc"
multiprocessor = None

data_loader = DataLoader(data_dir=data_dir, file_signature=file_signature, multiprocessor=multiprocessor)

In [ ]:
data_loader.print_netcdf_structure(data_loader.file_paths[0])

In [ ]:
df = data_loader.read_multi_netcdf()

In [5]:
data_filter = DataFilter(raw_df=df)
df = data_filter.filter_turbine_data(status_codes=[1], availability_codes=[100], include_nan=True)

In [6]:
turbine_input_filepath = "/Users/ahenry/Documents/toolboxes/wind_forecasting/examples/inputs/ge_282_127.yaml"
farm_input_filepath = "/Users/ahenry/Documents/toolboxes/wind_forecasting/examples/inputs/gch_KP_v4.yaml"
data_inspector = DataInspector(df=df, turbine_input_filepath=turbine_input_filepath, farm_input_filepath=farm_input_filepath)

In [ ]:
# from types import MethodType
# data_inspector.plot_time_series = MethodType(plot_time_series, data_inspector)
data_inspector.plot_time_series(["wt073"])

In [ ]:
data_inspector.plot_scatter_wind_speed_power()